In [1]:
import pandas as pd
import numpy as np
import math
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error


spot = pd.read_csv('./data/spot/clarkson_data.csv', delimiter=';', parse_dates=['Date'], dayfirst=True)
pmx_forw = pd.read_csv('./data/ffa/PMAX_FFA.csv', delimiter=';', parse_dates=['Date'], dayfirst=True)
csz_forw = pd.read_csv('./data/ffa/CSZ_FFA.csv', delimiter=';', parse_dates=['Date'], dayfirst=True)
smx_forw = pd.read_csv('./data/ffa/SMX_FFA.csv', delimiter=';', parse_dates=['Date'], dayfirst=True)
#oecd_ip_dev = pd.read_csv('./data/other/oecd_daily.csv', parse_dates=['Date'], dayfirst=True)
#fleet_dev = pd.read_csv('./data/other/fleet_dev_daily.csv', parse_dates=['Date'], dayfirst=True)
eur_usd = pd.read_csv('./data/other/EUR_USD_historical.csv', parse_dates=['Date'], delimiter=";", dayfirst=True)
# Convert 'Last' column to numeric, replacing comma with dot for decimal point
eur_usd['Last'] = pd.to_numeric(eur_usd['Last'].str.replace(',', '.'), errors='coerce')


def pick_forw(key):
    if key == "PMX":
        return pmx_forw
    elif key == "CSZ":
        return csz_forw
    elif key == "SMX":
        return smx_forw

In [2]:
# Number of rounds based on the test set size and forecast horizon
num_rounds =  5  # Adjusted to ensure we don't exceed the test set
look_back = 10  # Adjust based on your temporal structure
hor = 3
scaler = MinMaxScaler()
s_col = "CSZ"
f_col = "1MON"
#fleet_col = "CSZ fleet"
forw = pick_forw(s_col)



# Ensure 'Date' columns are in datetime format for all datasets
#oecd_ip_dev['Date'] = pd.to_datetime(oecd_ip_dev['Date'])
#fleet_dev['Date'] = pd.to_datetime(fleet_dev['Date'])
eur_usd['Date'] = pd.to_datetime(eur_usd['Date'])
spot['Date'] = pd.to_datetime(spot['Date'])
pmx_forw['Date'] = pd.to_datetime(pmx_forw['Date'])
csz_forw['Date'] = pd.to_datetime(csz_forw['Date'])
smx_forw['Date'] = pd.to_datetime(smx_forw['Date'])


#prod_col = 'Ind Prod Excl Const VOLA'
eur_col = 'Last'

# Merge data frames on the Date column
data_combined = pd.merge(spot, forw, on='Date')
#data_combined = pd.merge(data_combined, oecd_ip_dev[['Date', prod_col]], on='Date', how='inner')
#data_combined = pd.merge(data_combined, fleet_dev[['Date', fleet_col]], on='Date', how='inner')
data_combined = pd.merge(data_combined, eur_usd[['Date', eur_col]], on='Date', how='inner')


# Filter out rows where the specified columns contain zeros or NA values
cols_to_check = [s_col, f_col, eur_col]
data_combined = data_combined.dropna(subset=cols_to_check)  # Drop rows where NA values are present in the specified columns
data_combined = data_combined[(data_combined[cols_to_check] != 0).all(axis=1)]  # Drop rows where 0 values are present in the specified columns


# Remove rows with NA or 0 in specific columns (assuming 'SMX' and '1Q' are column names in 'data_combined')
#data_combined = data_combined[(data_combined[s_col].notna() & data_combined[s_col] != 0) & (data_combined[f_col].notna() & data_combined[f_col] != 0)]

# Transform data to log levels
data_log_levels = pd.DataFrame()
data_log_levels["spot"] = np.log(data_combined[s_col])
data_log_levels["forwp"] = np.log(data_combined[f_col])
#data_log_levels[fleet_col] = np.log(data_combined[fleet_col])
#data_log_levels[prod_col] = np.log(data_combined[prod_col])
data_log_levels[eur_col] = np.log(data_combined[eur_col])

data_log_levels.index = data_combined["Date"]

split_index = math.floor(len(data_log_levels) * 0.8)
print(len(data_log_levels))





# Initialize dictionary to store MSE results for each model
mse_results = {
    'MLP_spot': [],
    'MLP_forwp': [],
    'MLP_POINT_spot': [],
    'MLP_POINT_forwp': [],    
    'LSTM_spot': [],
    'LSTM_forwp': [],
    'RW_spot': [],
    'RW_forwp': [],
}

def random_walk_predictions(training_data, testing_data):
    """
    Generates Random Walk predictions where the next value is assumed to be the last observed value.
    
    Parameters:
    - training_data: DataFrame containing the training data.
    - testing_data: DataFrame containing the test data.
    
    Returns:
    - predictions: Numpy array containing Random Walk predictions for the test set.
    """
    # Last observed values from the training set
    last_observed_spot = training_data['spot'].iloc[-1]
    last_observed_forwp = training_data['forwp'].iloc[-1]
    
    # Create an array of predictions, each one equal to the last observed values
    predictions = np.tile([last_observed_spot, last_observed_forwp], (len(testing_data), 1))
    
    return predictions


def create_even_odd_array(arr):
    """
    Returns an array where the first column contains values from even positions
    and the second column contains values from odd positions of the original array.
    """
    return arr.reshape(-1, 2)


def create_dataset(dataset, look_back=10, hor=1, is_test=False, exog_col=None):
    X, Y = [], []
    if is_test:  # for test data, we just need the last entry for 1-step ahead forecast
        X = dataset[-1:,:,]
        return np.array(X), None
    else:
        for i in range(look_back, len(dataset) - hor + 1):
            X.append(dataset[i - look_back:i])
            if exog_col is not None:
                # Exclude specified columns from Y
                y = np.delete(dataset[i:i + hor], exog_col, axis=1)
            else:
                y = dataset[i:i + hor]
            Y.append(y)
    return np.array(X), np.array(Y)


def create_dataset_point(dataset, look_back=10, hor=1, is_test=False, exog_col=None):
    X, Y = [], []
    if is_test:  # for test data, we just need the last entry for 1-step ahead forecast
        X = dataset[-1:,:,:]
        return np.array(X), None
    else:
        for i in range(look_back, len(dataset) - hor + 1):
            X.append(dataset[i - look_back:i])
            if exog_col is not None:
                # Exclude specified columns and take only the point at 'hor' for Y
                y = np.delete(dataset[i + hor - 1: i + hor], exog_col, axis=1)
            else:
                y = dataset[i + hor - 1: i + hor]
            Y.append(y)
    return np.array(X), np.array(Y)



# Adjust train and test sets for each forecast round
for round in range(1, num_rounds + 1):
    print("Round", round)
    # Define new split point for each round
    split_index = split_index +  hor
    print(split_index)
    
    # Update train and test sets
    train = data_log_levels.iloc[:split_index]
    test = data_log_levels.iloc[split_index:split_index+hor]

    #Scale train set
    train_scal = scaler.fit_transform(train)

    trainX, trainY = create_dataset(train_scal, look_back=look_back, hor=hor, exog_col=[2])
    
    
    
    
    # Create and fit the MLP model

    trainX_flat = trainX.reshape(trainX.shape[0], -1)
    trainY_flat = trainY.reshape(trainY.shape[0], -1)

    model_mlp = Sequential()
    model_mlp.add(Input(shape=(trainX_flat.shape[1],)))  # Add Input layer
    model_mlp.add(Dense(32, activation='relu'))
    model_mlp.add(Dense(trainY_flat.shape[1], activation="linear"))
    model_mlp.compile(loss='mean_squared_error', optimizer='adam')
    model_mlp.fit(trainX_flat, trainY_flat, epochs=2, batch_size=1, verbose=1)


    # Make predictions
    trainPredict_scal_flat = model_mlp.predict(trainX_flat)
    testX = train_scal[-look_back:].reshape(1, look_back, train_scal.shape[1])
    #testX, _ = create_dataset(trainX, look_back=look_back, is_test=True)
    testX_flat = testX.reshape(testX.shape[0], -1)
    testPredict_scal_flat = model_mlp.predict(testX_flat)
    testPredict_scal = create_even_odd_array(testPredict_scal_flat)

    # Invert predictions
    #testPredict_mlp = scaler.inverse_transform(testPredict_scal)
    placeholder_exog = np.zeros((testPredict_scal.shape[0], 1))  # Assuming zero as placeholder
    testPredict_expanded = np.hstack((testPredict_scal, placeholder_exog))
    # Step 2: Inverse transform the expanded array
    testPredict_mlp_expanded = scaler.inverse_transform(testPredict_expanded)
    # Step 3: Extract the original predictions (now inversely scaled)
    testPredict_mlp = testPredict_mlp_expanded[:, :2]  # Assuming the first two columns are what you need

    # Calculate mean squared error
    testScore = mean_squared_error(test["spot"], testPredict_mlp[:,0])
    testScoreForw = mean_squared_error(test["forwp"], testPredict_mlp[:,1])
    print('Test Score spot MLP: %.5f MSE' % (testScore))
    print('Test Score forw MLP: %.5f MSE' % (testScoreForw))
    
    




    # Define the LSTM model
    #model_lstm = Sequential()
    #model_lstm.add(LSTM(units=50, return_sequences=True, input_shape=(trainX.shape[1], trainX.shape[2])))
    #model_lstm.add(LSTM(units=50))
    #model_lstm.add(Dense(trainY_flat.shape[1], activation='linear'))  # Assuming multi-step forecasting

    #model_lstm.compile(loss='mean_squared_error', optimizer='adam')
    #model_lstm.fit(trainX, trainY_flat, epochs=30, batch_size=1, verbose=0)
    


    # Define the LSTM model with the Input layer
    model_lstm = Sequential()
    model_lstm.add(Input(shape=(trainX.shape[1], trainX.shape[2])))
    model_lstm.add(LSTM(units=50, return_sequences=True))
    model_lstm.add(LSTM(units=50))
    model_lstm.add(Dense(trainY_flat.shape[1], activation='linear'))

    model_lstm.compile(loss='mean_squared_error', optimizer='adam')
    model_lstm.fit(trainX, trainY_flat, epochs=2, batch_size=1, verbose=1)



    # Prepare the last sequence from the training set as the input for the first prediction
    #testX_last_sequence = train_scal[-look_back:].reshape(1, look_back, train_scal.shape[1])

    # Make predictions
    testPredict_scal_flat = model_lstm.predict(testX)

    # Since you're predicting `hor` steps ahead, you might need to adjust the code to generate
    # multiple steps if your LSTM model is set up for single-step predictions.
    # For simplicity, this example directly uses the LSTM output for multi-step predictions.

    # Invert scaling
    testPredict_scal = create_even_odd_array(testPredict_scal_flat)
    # Expand the array to include the exogenous column for inverse scaling
    # Here, placeholder_exog needs to be reshaped or sliced appropriately to match the testPredict_scal's number of rows
    placeholder_exog = np.zeros((testPredict_scal.shape[0], 1))  # Example placeholder

    # Combine LSTM predictions with the placeholder for the exogenous variable
    testPredict_expanded = np.hstack((testPredict_scal, placeholder_exog))

    # Inverse transform the expanded array to scale back to original scale
    testPredict_lstm_expanded = scaler.inverse_transform(testPredict_expanded)

    # Extract only the inversely scaled predictions, excluding the exogenous variable's placeholder column
    testPredict_lstm = testPredict_lstm_expanded[:, :2]  # Assuming the first two columns are the endogenous variables you predicted
    #testPredict_lstm = scaler.inverse_transform(testPredict_scal)


    # Calculate and print MSE for each target
    testScore_spot_lstm = mean_squared_error(test["spot"].iloc[:hor].values, testPredict_lstm[:,0])
    testScore_forw_lstm = mean_squared_error(test["forwp"].iloc[:hor].values, testPredict_lstm[:,1])
    print('LSTM Test Score spot: %.5f MSE' % (testScore_spot_lstm))
    print('LSTM Test Score forw: %.5f MSE' % (testScore_forw_lstm))
    
    
    
    # Point forecasts

    # Store the original last look_back points to initiate prediction for each model
    original_last_points = train_scal[-look_back:].reshape(1, look_back, train_scal.shape[1])

    # Placeholder to store combined predictions for all horizons
    combined_testPredict_point = np.zeros((1, hor * 2))  # Multiply by 2 as each step predicts 2 variables

    for step_ahead in range(1, hor + 1):
        # Create dataset for current horizon
        trainX, trainY = create_dataset_point(train_scal, look_back=look_back, hor=step_ahead, exog_col=[2])

        # Flatten input and output
        trainX_flat = trainX.reshape(trainX.shape[0], -1)
        trainY_flat = trainY.reshape(trainY.shape[0], -1)

        # Define and fit the MLP model for the current horizon
        model_mlp_point = Sequential()
        model_mlp_point.add(Input(shape=(trainX_flat.shape[1],)))
        model_mlp_point.add(Dense(32, activation='relu'))
        model_mlp_point.add(Dense(trainY_flat.shape[1], activation="linear"))
        model_mlp_point.compile(loss='mean_squared_error', optimizer='adam')
        model_mlp_point.fit(trainX_flat, trainY_flat, epochs=2, batch_size=1, verbose=1)

        # Use the original last points to predict the step ahead
        testX_flat = original_last_points.reshape(original_last_points.shape[0], -1)
        testPredict_scal_flat_point = model_mlp_point.predict(testX_flat)
        
        # Store the prediction for the current horizon
        combined_testPredict_point[0, (step_ahead - 1) * 2: step_ahead * 2] = testPredict_scal_flat_point

    # Invert predictions
    testPredict_scal = create_even_odd_array(combined_testPredict_point)
    
    # Invert predictions
    #testPredict_mlp = scaler.inverse_transform(testPredict_scal)
    placeholder_exog = np.zeros((testPredict_scal.shape[0], 1))  # Assuming zero as placeholder
    testPredict_expanded = np.hstack((testPredict_scal, placeholder_exog))
    # Step 2: Inverse transform the expanded array
    testPredict_mlp_point_expanded = scaler.inverse_transform(testPredict_expanded)
    # Step 3: Extract the original predictions (now inversely scaled)
    testPredict_mlp_point = testPredict_mlp_point_expanded[:, :2]  # Assuming the first two columns are what you need
    


    # Reshape testPredict_mlp_point if necessary to match the test target shape and calculate error metrics

    # Calculate and print MSE for each target
    testScore_spot_mlp_point = mean_squared_error(test["spot"].iloc[:hor].values, testPredict_mlp_point[:,0])
    testScore_forw_mlp_point = mean_squared_error(test["forwp"].iloc[:hor].values, testPredict_mlp_point[:,1])
    print('MLP POINT Test Score spot: %.5f MSE' % (testScore_spot_mlp_point))
    print('MLP POINT Test Score forw: %.5f MSE' % (testScore_forw_mlp_point))
    
    
    


    # Random Walk Predictions for comparison
    rw_predictions = random_walk_predictions(train, test)
    
    # Calculate and append MSE for each model for this round
    mse_results['MLP_spot'].append(mean_squared_error(test["spot"], testPredict_mlp[:, 0]))
    mse_results['MLP_forwp'].append(mean_squared_error(test["forwp"], testPredict_mlp[:, 1]))
    
     # Calculate and append MSE for each model for this round
    mse_results['MLP_POINT_spot'].append(mean_squared_error(test["spot"], testPredict_mlp_point[:, 0]))
    mse_results['MLP_POINT_forwp'].append(mean_squared_error(test["forwp"], testPredict_mlp_point[:, 1]))

    mse_results['LSTM_spot'].append(mean_squared_error(test["spot"], testPredict_lstm[:, 0]))
    mse_results['LSTM_forwp'].append(mean_squared_error(test["forwp"], testPredict_lstm[:, 1]))
    
    

    mse_results['RW_spot'].append(mean_squared_error(test["spot"], rw_predictions[:, 0]))
    mse_results['RW_forwp'].append(mean_squared_error(test["forwp"], rw_predictions[:, 1]))



3597
Round 1
2880
Epoch 1/2
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 1s 275us/step - loss: 0.0171
Epoch 2/2
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 1s 273us/step - loss: 0.0018
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 343us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Test Score spot MLP: 0.01784 MSE
Test Score forw MLP: 0.00713 MSE
Epoch 1/2
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.0084
Epoch 2/2
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - loss: 0.0020
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
LSTM Test Score spot: 0.00107 MSE
LSTM Test Score forw: 0.00470 MSE
Epoch 1/2
2870/2870 ━━━━━━━━━━━━━━━━━━━━ 1s 278us/step - loss: 0.0065
Epoch 2/2
2870/2870 ━━━━━━━━━━━━━━━━━━━━ 1s 272us/step - loss: 0.0011
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Epoch 1/2
2869/2869 ━━━━━━━━━━━━━━━━━━━━ 1s 277us/step - loss: 0.0209
Epoch 2/2
2869/2869 ━━━━━━━━━━━━━━━━━━━━ 1s 272us/step - loss: 0.0019
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Epoch 1/2
2868/2868 ━━━━━━━━━━━━━━━━━━━━ 1s 277us/step - loss: 0.0096
Epoch 2/2
2868/2868 ━━━━━━━━━

Mean for MLP_spot: 1.1739799029579019
Mean for MLP_forwp: 1.6197907482840133
Mean for MLP_POINT_spot: 1.1914217318326712
Mean for MLP_POINT_forwp: 1.7970986584315993
Mean for LSTM_spot: 0.4566449139397142
Mean for LSTM_forwp: 2.9257590007351713
Mean for RW_spot: 0.6596570131445423
Mean for RW_forwp: 1.0305957479552907
